In [1]:
import swiftest
from astroquery.jplhorizons import Horizons
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sim_gr = swiftest.Simulation(tstop=1000.0, dt=0.005, tstep_out=1.0, integrator="whm", init_cond_format="XV", general_relativity=True, param_file="param.gr.in", output_file_name="bin.gr.nc")
sim_gr.add_solar_system_body(["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"])

Creating the Sun as a central body
Fetching ephemerides data for Mercury from JPL/Horizons
Fetching ephemerides data for Venus from JPL/Horizons
Fetching ephemerides data for Earth from JPL/Horizons
Fetching ephemerides data for Mars from JPL/Horizons
Fetching ephemerides data for Jupiter from JPL/Horizons
Fetching ephemerides data for Saturn from JPL/Horizons
Fetching ephemerides data for Uranus from JPL/Horizons
Fetching ephemerides data for Neptune from JPL/Horizons
Writing initial conditions to file init_cond.nc
Writing parameter inputs to file /home/daminton/git_debug/swiftest/examples/whm_gr_test/param.gr.in


<xarray.Dataset>
Dimensions:        (name: 9, time: 1)
Coordinates:
  * name           (name) <U32 'Sun' 'Mercury' 'Venus' ... 'Uranus' 'Neptune'
  * time           (time) float64 0.0
Data variables: (12/14)
    particle_type  (name) <U32 'Central Body' 'Massive Body' ... 'Massive Body'
    id             (name) int64 0 1 2 3 4 5 6 7 8
    Gmass          (time, name) float64 39.48 6.554e-06 ... 0.001724 0.002034
    radius         (time, name) float64 0.00465 1.631e-05 ... 0.0001646
    j2rp2          (time, name) float64 4.754e-12 nan nan nan ... nan nan nan
    j4rp4          (time, name) float64 -2.247e-18 nan nan nan ... nan nan nan
    ...             ...
    xhz            (time, name) float64 nan -0.002529 ... -0.0407 -0.7287
    vhx            (time, name) float64 nan -9.241 3.454 ... -1.315 -0.08755
    vhy            (time, name) float64 nan 7.755 6.477 ... 1.932 0.5372 1.149
    vhz            (time, name) float64 nan 1.481 -0.1103 ... 0.01905 -0.02168
    ntp            (time) int64 0
    npl            (time) int64 8

In [3]:
sim_nogr = swiftest.Simulation(tstop=1000.0, dt=0.005, tstep_out=1.0, integrator="whm", init_cond_format="XV", general_relativity=False,param_file="param.nogr.in", output_file_name="bin.nogr.nc")
sim_nogr.add_solar_system_body(["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"])

Creating the Sun as a central body
Fetching ephemerides data for Mercury from JPL/Horizons
Fetching ephemerides data for Venus from JPL/Horizons
Fetching ephemerides data for Earth from JPL/Horizons
Fetching ephemerides data for Mars from JPL/Horizons
Fetching ephemerides data for Jupiter from JPL/Horizons
Fetching ephemerides data for Saturn from JPL/Horizons
Fetching ephemerides data for Uranus from JPL/Horizons
Fetching ephemerides data for Neptune from JPL/Horizons
Writing initial conditions to file init_cond.nc
Writing parameter inputs to file /home/daminton/git_debug/swiftest/examples/whm_gr_test/param.nogr.in


<xarray.Dataset>
Dimensions:        (name: 9, time: 1)
Coordinates:
  * name           (name) <U32 'Sun' 'Mercury' 'Venus' ... 'Uranus' 'Neptune'
  * time           (time) float64 0.0
Data variables: (12/14)
    particle_type  (name) <U32 'Central Body' 'Massive Body' ... 'Massive Body'
    id             (name) int64 0 1 2 3 4 5 6 7 8
    Gmass          (time, name) float64 39.48 6.554e-06 ... 0.001724 0.002034
    radius         (time, name) float64 0.00465 1.631e-05 ... 0.0001646
    j2rp2          (time, name) float64 4.754e-12 nan nan nan ... nan nan nan
    j4rp4          (time, name) float64 -2.247e-18 nan nan nan ... nan nan nan
    ...             ...
    xhz            (time, name) float64 nan -0.002529 ... -0.0407 -0.7287
    vhx            (time, name) float64 nan -9.241 3.454 ... -1.315 -0.08755
    vhy            (time, name) float64 nan 7.755 6.477 ... 1.932 0.5372 1.149
    vhz            (time, name) float64 nan 1.481 -0.1103 ... 0.01905 -0.02168
    ntp            (time) int64 0
    npl            (time) int64 8

In [4]:
%%capture
#Capture the output of the run so it doesn't take up the whole Notebook
sim_gr.run()
sim_nogr.run()

In [5]:
# Get the start and end date of the simulation so we can compare with the real solar system
start_date = sim_gr.ephemeris_date
tstop_d = sim_gr.param['TSTOP'] * sim_gr.param['TU2S'] / swiftest.JD2S

stop_date = (datetime.datetime.fromisoformat(start_date) + datetime.timedelta(days=tstop_d)).isoformat()

In [6]:
#Get the ephemerides of Mercury for the same timeframe as the simulation
obj = Horizons(id='1', location='@sun',
               epochs={'start':start_date, 'stop':stop_date,
                       'step':'1y'})
el = obj.elements()
t = (el['datetime_jd']-el['datetime_jd'][0]) / 365.25
varpi_obs = el['w'] + el['Omega']

In [7]:
# Compute the longitude of the periapsis
sim_gr.data['varpi'] = np.mod(sim_gr.data['omega'] + sim_gr.data['capom'],360)
sim_nogr.data['varpi'] = np.mod(sim_nogr.data['omega'] + sim_nogr.data['capom'],360)

/home/daminton/.conda/envs/cent7/2020.11-py38/swiftest/lib/python3.8/site-packages/xarray/core/computation.py:733: RuntimeWarning: invalid value encountered in remainder
  result_data = func(*input_data)


In [12]:
varpisim_gr= sim_gr.data['varpi'].sel(name="Mercury")
varpisim_nogr= sim_nogr.data['varpi'].sel(name="Mercury")
tsim = sim_gr.data['time']

In [13]:
dvarpi_gr = np.diff(varpisim_gr)  * 3600 * 100
dvarpi_nogr = np.diff(varpisim_nogr)  * 3600 * 100 
dvarpi_obs = np.diff(varpi_obs) / np.diff(t) * 3600 * 100 

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, varpi_obs, label="JPL Horizons")
ax.plot(tsim, varpisim_gr, label="Swiftest WHM GR")
ax.plot(tsim, varpisim_nogr, label="Swiftest WHM No GR")
ax.set_xlabel('Time (y)')
ax.set_ylabel('Mercury $\\varpi$ (deg)')
ax.legend()
print('Mean precession rate for Mercury long. peri. (arcsec/100 y)')
print(f'JPL Horizons         : {np.mean(dvarpi_obs)}')
print(f'Swiftest No GR       : {np.mean(dvarpi_nogr)}')
print(f'Swiftest GR          : {np.mean(dvarpi_gr)}')
print(f'Obs - Swiftest GR    : {np.mean(dvarpi_obs - dvarpi_gr)}')
print(f'Obs - Swiftest No GR : {np.mean(dvarpi_obs - dvarpi_nogr)}')